## Import libraries

In [1]:
# !pip install tensorly

import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from math import ceil
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
import torch.nn.init as init

In [2]:
"""
5 runs of 50 epochs, seed = 10, 20, 30, 40, 50;
validation accuracies: 0.9492, 0.9457, 0.9463, 0.9439, 0.9455
"""
#from __future__ import print_function, division
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import balanced_accuracy_score

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

dtype = torch.float64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


## train data processing

In [3]:
df_train_1 = pd.read_csv('/home/c/cl237/Datasets/Flare_LSTM_dataset/M/normalized_training.csv')
df_train_2 = pd.read_csv('/home/c/cl237/Datasets/Flare_LSTM_dataset/M/normalized_validation.csv')
# concat train and validation into train. we don't need validation set
df_train = df_train_1.append(df_train_2).reset_index(drop=True)

# undersampling for "negative" samples
negative_df = df_train[df_train['label'] == 'Negative']
positive_df = df_train[df_train['label'] == 'Positive']

# Get number of "positive" samples
num_positive = len(positive_df)

# take the same number of "negative" samples as there are "positive" samples
balanced_negative_df = negative_df.sample(n=num_positive, random_state=10086)

df_train_balanced = pd.concat([positive_df, balanced_negative_df])
print(df_train_balanced['label'].value_counts())

Positive    4057
Negative    4057
Name: label, dtype: int64


In [4]:
pd_X_train = df_train_balanced.iloc[:, 5:]
pd_y_train = df_train_balanced.iloc[:, 0]

N = len(pd_X_train)
K = 2

pd_X_train = pd_X_train.values
X_train = torch.tensor(pd_X_train, dtype=dtype, device=device)
X_train = torch.t(X_train)

# scaler = MinMaxScaler()
# x = pd_X_train.values
# x_scaled = scaler.fit_transform(x)
# X_train = torch.tensor(x_scaled, dtype=dtype, device=device)
# X_train = torch.t(X_train)

# Initialize the LabelEncoder
encoder = LabelEncoder()
# Fit and transform the y values
y = encoder.fit_transform(pd_y_train.values)
y_train=torch.tensor(y, dtype=torch.long, device=device)
y_train = torch.flatten(y_train)

y_one_hot = torch.zeros(N, K, device=device).scatter_(1, y_train.unsqueeze(1), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)

print(list(encoder.classes_))

['Negative', 'Positive']


## test data processing

In [5]:
df_test = pd.read_csv('/home/c/cl237/Datasets/Flare_LSTM_dataset/M/normalized_testing.csv')

# Undersampling for "negative" samples
#negative_df = df_test[df_test['label'] == 'Negative']
#positive_df = df_test[df_test['label'] == 'Positive']

# Get number of "positive" samples
#num_positive = len(positive_df)

# Take the same number of "negative" samples as there are "positive" samples
#balanced_negative_df = negative_df.sample(n=num_positive, random_state=10086)

#df_test_balanced = pd.concat([positive_df, balanced_negative_df])
#print(df_test_balanced['label'].value_counts())

In [6]:
pd_X_test = df_test.iloc[:, 5:]
pd_y_test = df_test.iloc[:, 0]

N_test = len(pd_X_test)
K = 2

pd_X_test = pd_X_test.values
X_test = torch.tensor(pd_X_test, dtype=dtype, device=device)
X_test = torch.t(X_test)
# x = pd_X_test.values
# x_scaled = scaler.transform(x)  # only transform x, don't fit the scaler again
# X_test = torch.tensor(x_scaled, dtype=dtype, device=device)
# X_test = torch.t(X_test)


# Initialize the LabelEncoder
encoder = LabelEncoder()
# Fit and transform the y values
y = encoder.fit_transform(pd_y_test.values)
y_test=torch.tensor(y, dtype=torch.long, device=device)
y_test = torch.flatten(y_test)

y_test_one_hot = torch.zeros(N_test, K, device=device).scatter_(1, y_test.unsqueeze(1), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)

In [7]:
print(df_test['label'].value_counts())

Negative    43411
Positive     1278
Name: label, dtype: int64


## Main algorithm

### Define functions for updating blocks

In [8]:
# def updateMask(W, sparsity):
#     torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy()
#     Mask
#     Wsparse
#     return Mask, Wsparse

# Wsquare = torch.square(W3)
# Threshold = torch.quantile(torch.reshape(Wsquare, (-1,)), 0.5, interpolation='linear')
# Wsparse = W3
# Wsparse[Wsquare < Threshold] =  0

# Wsparse

In [9]:
def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device)
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    return Vstar

In [10]:
def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [11]:
def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [12]:
def updateWbsparse(U, V, W, b,  W_tensor2matrix, alpha, rho,tau):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

### Define the proximal operator of the ReLU activation function

In [13]:
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

### Effective Sparsity

In [14]:
def process_weights(W1, W2, W3):
    # Clone the tensors to keep the originals unchanged
    W1, W2, W3 = W1.clone(), W2.clone(), W3.clone()

    while True:
        # Store a copy of the current weights
        old_W1, old_W2, old_W3 = W1.clone(), W2.clone(), W3.clone()

        # Check all rows of W1
        # If all values in a row of W1 are 0, set corresponding column in W2 to 0
        zero_rows_W1 = torch.all(W1 == 0, dim=1)
        W2[:, zero_rows_W1] = 0

        # Check all columns of W2
        # If all values in a column of W2 are 0, set corresponding row in W1 to 0
        zero_cols_W2 = torch.all(W2 == 0, dim=0)
        W1[zero_cols_W2, :] = 0

        # Check all rows of W2
        # If all values in a row of W2 are 0, set corresponding column in W3 to 0
        zero_rows_W2 = torch.all(W2 == 0, dim=1)
        W3[:, zero_rows_W2] = 0

        # Check if matrices are unchanged
        if torch.equal(W1, old_W1) and torch.equal(W2, old_W2) and torch.equal(W3, old_W3):
            break

    return W1, W2, W3

### Parameter initialization

In [16]:
#df = pd.DataFrame()
#df.to_csv('/home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/differernt gamma rho/result.csv')

niter = 1000
sparsity = 0.9

tau = 200
alpha = 1
rho = 1
gamma = 100

        

# Do something with these parameters
print(sparsity, tau, gamma, rho, alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)
bacc_train = np.empty(niter)
bacc_test = np.empty(niter)
true_sparsity = np.empty(niter)
effective_sparsity = np.empty(niter)

results = torch.zeros(1, 9, niter)

for Out_iter in range(1):
    rank_initial = 700
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 40
    d1 =  300
    d2 =  100
    d3 = K # Layers: input + 2 hidden + output

    W1 = init.uniform_(torch.empty(d1, d0, device=device, dtype=dtype), a=-0.01, b=0.01)
    b1 = 0*torch.ones(d1, 1, dtype=torch.float64,device=device)
    W1square = torch.square(W1)
    Threshold = torch.quantile(torch.reshape(W1square, (-1,)), sparsity, interpolation='linear')
    W1sparse = W1
    W1sparse[W1square < Threshold] =  0

    W2 = init.uniform_(torch.empty(d2, d1, device=device, dtype=dtype), a=-0.01, b=0.01)
    b2 = 0*torch.ones(d2, 1, dtype=torch.float64,device=device)
    W2square = torch.square(W2)
    Threshold = torch.quantile(torch.reshape(W2square, (-1,)), sparsity, interpolation='linear')
    W2sparse = W2
    W2sparse[W2square < Threshold] =  0


    W3 = init.uniform_(torch.empty(d3, d2, device=device, dtype=dtype), a=-0.01, b=0.01)
    b3 = 0*torch.ones(d3, 1, dtype=torch.float64,device=device)
    # W3square = torch.square(W3)
    # Threshold = torch.quantile(torch.reshape(W3square, (-1,)), sparsity, interpolation='linear')
    # W3sparse = W3
    # W3sparse[W3square < Threshold] =  0


    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = U3
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4


    print('Train on', N, 'samples, validate on', N_test, 'samples')

    for k in range(niter):

        start = time.time()

        # update V4
        V3 = (y_one_hot + gamma*U3 + alpha*V3)/(1 + gamma + alpha)

        # update U4
        U3 = (gamma*V3 + rho*(torch.mm(W3,V2) + b3.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W3, b3 = updateWb_org(U3,V2,W3,b3, alpha,rho)
        # W3square = torch.square(W3)
        # Threshold = torch.quantile(torch.reshape(W3square, (-1,)), sparsity, interpolation='linear')
        # W3sparse = W3
        # W3sparse[W3square < Threshold] =  0
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2

        W2, b2 = updateWbsparse(U2,V1,W2,b2,W2sparse, alpha,rho,tau)
        W2square = torch.square(W2)
        Threshold = torch.quantile(torch.reshape(W2square, (-1,)), sparsity, interpolation='linear')
        W2sparse = W2
        W2sparse[W2square < Threshold] =  0

        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWbsparse(U1,X_train,W1,b1,W1sparse, alpha,rho,tau)
        W1square = torch.square(W1)
        Threshold = torch.quantile(torch.reshape(W1square, (-1,)), sparsity, interpolation='linear')
        W1sparse = W1
        W1sparse[W1square < Threshold] =  0

        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1sparse, X_train)).double()
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N),  W2sparse, a1_train)).double()
        pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1sparse, X_test)).double()
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test),  W2sparse, a1_test)).double()
        pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)


        loss1[k] = gamma/2*torch.pow(torch.dist(V3,y_one_hot,2),2).cpu().numpy()
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1sparse, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N),  W2sparse, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,U3,2),2).cpu().numpy() \
        + tau/2*torch.pow(torch.dist(W1,W1sparse,2),2).cpu().numpy()\
        + tau/2*torch.pow(torch.dist(W2,W2sparse,2),2).cpu().numpy()\
        # + tau/2*torch.pow(torch.dist(W3,W3sparse,2),2).cpu().numpy()

        # compute training accuracy
        correct_train = pred == y_train
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())


        pred_train_np = pred.cpu().numpy()
        y_train_np = y_train.cpu().numpy()
        pred_test_np = pred_test.cpu().numpy()
        y_test_np = y_test.cpu().numpy()

        bacc_train[k] = balanced_accuracy_score(y_train_np, pred_train_np)
        bacc_test[k] = balanced_accuracy_score(y_test_np, pred_test_np)

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # sparsity
        num_zeros_W1 = torch.sum(W1 == 0).item()
        num_zeros_W2 = torch.sum(W2 == 0).item()
        num_zeros_W3 = torch.sum(W3 == 0).item()
        total_zeros_old = num_zeros_W1 + num_zeros_W2 + num_zeros_W3
        total_weights = d0*d1+d1*d2+d2*d3
        true_sparsity[k] = total_zeros_old / total_weights

        new_W1, new_W2, new_W3 = process_weights(W1, W2, W3)
        num_zeros_W1_new = torch.sum(new_W1 == 0).item()
        num_zeros_W2_new = torch.sum(new_W2 == 0).item()
        num_zeros_W3_new = torch.sum(new_W3 == 0).item()
        total_zeros = num_zeros_W1_new + num_zeros_W2_new + num_zeros_W3_new
        total_weights = d0*d1+d1*d2+d2*d3
        effective_sparsity[k] = total_zeros / total_weights


        # print results
        print('Epoch', k + 1, '/', niter, '\n',
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k],
              '-', 'bacc_train:', bacc_train[k], '-', 'bacc_test:', bacc_test[k],'-', 'true_sparsity:', true_sparsity[k],
             '-', 'effective_sparsity:', effective_sparsity[k])


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    results[Out_iter,5,:] = torch.tensor(bacc_train)
    results[Out_iter,6,:] = torch.tensor(bacc_test)
    results[Out_iter,7,:] = torch.tensor(true_sparsity)
    results[Out_iter,8,:] = torch.tensor(effective_sparsity)

#             df=pd.read_csv('/home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/differernt gamma rho/result.csv')
#             new_row = {
#                         'tau': tau, 
#                         'gamma': gamma, 
#                         'rho': rho, 
#                         'alpha': alpha,
#                         'loss1': loss1[niter-1], 
#                         'loss2': loss2[niter-1], 
#                         'accuracy_train': accuracy_train[niter-1],
#                         'accuracy_test': accuracy_test[niter-1], 
#                         'max_accuracy_train': max(accuracy_train),
#                         'max_accuracy_test': max(accuracy_test),
#                         'time': time1[niter-1], 
#                         'BACC_train': bacc_train[niter-1],
#                         'BACC_test': bacc_test[niter-1],
#                         'max_BACC_train': max(bacc_train),
#                         'max_BACC_test': max(bacc_test),
#                         'Sparsity': sparsity,
#                         'seed' : seed
#                     }

#             df=df.append(new_row,ignore_index=True)
#             df.to_csv('/home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/differernt gamma rho/result.csv',index=False)

filename="UniformScaled_" + "niter_"+ str(niter) + "Sparsity_" + str(sparsity) +"tau_" + str(tau) + "gamma_" + str(gamma) + \
                "rho_" + str(rho) + "alpha_" + str(alpha)+ ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/different init/
savemat (filename, {'results': torch.Tensor.numpy(results)})

0.9 200 100 1 1
Train on 8114 samples, validate on 44689 samples
Epoch 1 / 1000 
 - time: 0.7295775413513184 - sq_loss: 397755.41356882214 - tot_loss: 397755.7515115086 - acc: 0.5 - val_acc: 0.028597641477768578 - bacc_train: 0.5 - bacc_test: 0.5 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 2 / 1000 
 - time: 0.02849102020263672 - sq_loss: 390070.2274626472 - tot_loss: 390071.484676654 - acc: 0.5 - val_acc: 0.028597641477768578 - bacc_train: 0.5 - bacc_test: 0.5 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 3 / 1000 
 - time: 0.028199195861816406 - sq_loss: 382602.5768030275 - tot_loss: 382605.2118149946 - acc: 0.5 - val_acc: 0.028597641477768578 - bacc_train: 0.5 - bacc_test: 0.5 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 4 / 1000 
 - time: 0.028158187866210938 - sq_loss: 375341.317144702 - tot_loss: 375345.65084919165 - acc: 0.5001232437761893 - val_acc: 0.02879903331916

Epoch 32 / 1000 
 - time: 0.027875185012817383 - sq_loss: 219213.70031182579 - tot_loss: 219214.38288269076 - acc: 0.80564456494947 - val_acc: 0.8003983083085323 - bacc_train: 0.8056445649494701 - bacc_test: 0.8630863988844263 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 33 / 1000 
 - time: 0.027258634567260742 - sq_loss: 214988.85840292714 - tot_loss: 214989.54562858184 - acc: 0.8066305151589844 - val_acc: 0.8034639396719551 - bacc_train: 0.8066305151589845 - bacc_test: 0.8642846214706044 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 34 / 1000 
 - time: 0.02697467803955078 - sq_loss: 210845.1381384029 - tot_loss: 210845.82973352718 - acc: 0.8078629529208775 - val_acc: 0.806507194164112 - bacc_train: 0.8078629529208774 - bacc_test: 0.8654713262387179 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 35 / 1000 
 - time: 0.02682185173034668 - sq_loss: 206781.00948456273 - tot_loss:

Epoch 64 / 1000 
 - time: 0.02756643295288086 - sq_loss: 117552.66796953283 - tot_loss: 117553.39848207217 - acc: 0.8171062361350752 - val_acc: 0.850007831904943 - bacc_train: 0.8171062361350752 - bacc_test: 0.8791284393169065 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 65 / 1000 
 - time: 0.027240753173828125 - sq_loss: 115284.73511910863 - tot_loss: 115285.46527848694 - acc: 0.8168597485826966 - val_acc: 0.8509029067555774 - bacc_train: 0.8168597485826966 - bacc_test: 0.8792094335508236 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 66 / 1000 
 - time: 0.028280973434448242 - sq_loss: 113060.52923447457 - tot_loss: 113061.25911241569 - acc: 0.8177224550160217 - val_acc: 0.8516637203786167 - bacc_train: 0.8177224550160217 - bacc_test: 0.8792213208763535 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 67 / 1000 
 - time: 0.03740835189819336 - sq_loss: 110879.20920696137 - tot_lo

Epoch 95 / 1000 
 - time: 0.031017780303955078 - sq_loss: 64256.51574837666 - tot_loss: 64257.24914700711 - acc: 0.8228986936159723 - val_acc: 0.866589093512945 - bacc_train: 0.8228986936159723 - bacc_test: 0.8823470836614289 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 96 / 1000 
 - time: 0.027227401733398438 - sq_loss: 63016.552532469526 - tot_loss: 63017.287241350255 - acc: 0.8228986936159723 - val_acc: 0.8668352390968694 - bacc_train: 0.8228986936159723 - bacc_test: 0.882473779660139 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 97 / 1000 
 - time: 0.04511094093322754 - sq_loss: 61800.513653084556 - tot_loss: 61801.24969280961 - acc: 0.8227754498397831 - val_acc: 0.867059007809528 - bacc_train: 0.8227754498397831 - bacc_test: 0.8822092393521197 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 98 / 1000 
 - time: 0.02687358856201172 - sq_loss: 60607.93799827645 - tot_loss: 60

Epoch 126 / 1000 
 - time: 0.026944398880004883 - sq_loss: 35121.49855946515 - tot_loss: 35122.43003307446 - acc: 0.8280749322159231 - val_acc: 0.8769272080377721 - bacc_train: 0.8280749322159231 - bacc_test: 0.8793145088566252 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 127 / 1000 
 - time: 0.026819229125976562 - sq_loss: 34443.740837280675 - tot_loss: 34444.67099885735 - acc: 0.8286911510968696 - val_acc: 0.8769495849090381 - bacc_train: 0.8286911510968696 - bacc_test: 0.8797057451633545 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 128 / 1000 
 - time: 0.026836156845092773 - sq_loss: 33779.06279091486 - tot_loss: 33780.04627991588 - acc: 0.8294306137540054 - val_acc: 0.8722951956857392 - bacc_train: 0.8294306137540055 - bacc_test: 0.878828912960588 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 129 / 1000 
 - time: 0.02695488929748535 - sq_loss: 33127.21210613857 - tot_los

...

Epoch 853 / 1000 
 - time: 0.028272390365600586 - sq_loss: 0.05933403030964427 - tot_loss: 0.5372746997683387 - acc: 0.8460685235395613 - val_acc: 0.8762559018997964 - bacc_train: 0.8460685235395613 - bacc_test: 0.892259121417954 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 854 / 1000 
 - time: 0.026906251907348633 - sq_loss: 0.058656194520733264 - tot_loss: 0.5356029705399598 - acc: 0.8460685235395613 - val_acc: 0.876166394414733 - bacc_train: 0.8460685235395613 - bacc_test: 0.892213050145696 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 855 / 1000 
 - time: 0.02677011489868164 - sq_loss: 0.05798949047930693 - tot_loss: 0.5340178479147547 - acc: 0.8460685235395613 - val_acc: 0.876166394414733 - bacc_train: 0.8460685235395613 - bacc_test: 0.892213050145696 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 856 / 1000 
 - time: 0.026883602142333984 - sq_loss: 0.05733373817973867 - 

Epoch 885 / 1000 
 - time: 0.026912927627563477 - sq_loss: 0.042304209415786866 - tot_loss: 0.49199197527773914 - acc: 0.8469312299728864 - val_acc: 0.8758083644744792 - bacc_train: 0.8469312299728864 - bacc_test: 0.8905098911020043 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 886 / 1000 
 - time: 0.026871919631958008 - sq_loss: 0.04190053544663094 - tot_loss: 0.4909162882479341 - acc: 0.8469312299728864 - val_acc: 0.8758083644744792 - bacc_train: 0.8469312299728864 - bacc_test: 0.8905098911020043 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 887 / 1000 
 - time: 0.026920080184936523 - sq_loss: 0.04150318933736531 - tot_loss: 0.4898355213803008 - acc: 0.8469312299728864 - val_acc: 0.8757412338606816 - bacc_train: 0.8469312299728864 - bacc_test: 0.8900956191591459 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 888 / 1000 
 - time: 0.026861906051635742 - sq_loss: 0.0411120537954

Epoch 917 / 1000 
 - time: 0.02768087387084961 - sq_loss: 0.03204436045084767 - tot_loss: 0.4618641537532178 - acc: 0.8468079861966971 - val_acc: 0.8750475508514399 - bacc_train: 0.8468079861966971 - bacc_test: 0.8893588483104803 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 918 / 1000 
 - time: 0.02690410614013672 - sq_loss: 0.03179737609315106 - tot_loss: 0.4611262528288432 - acc: 0.8468079861966971 - val_acc: 0.8750475508514399 - bacc_train: 0.8468079861966971 - bacc_test: 0.8893588483104803 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 919 / 1000 
 - time: 0.026895523071289062 - sq_loss: 0.03155403348678239 - tot_loss: 0.46037086779084446 - acc: 0.8468079861966971 - val_acc: 0.8750027971089083 - bacc_train: 0.8468079861966971 - bacc_test: 0.8893358126743511 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 920 / 1000 
 - time: 0.026903152465820312 - sq_loss: 0.0313142742162155

Epoch 949 / 1000 
 - time: 0.0269930362701416 - sq_loss: 0.025680122543535033 - tot_loss: 0.4407775200683795 - acc: 0.8475474488538329 - val_acc: 0.8749580433663765 - bacc_train: 0.8475474488538328 - bacc_test: 0.8896924955268868 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 950 / 1000 
 - time: 0.026824235916137695 - sq_loss: 0.02552412442424744 - tot_loss: 0.4402686228684497 - acc: 0.8475474488538329 - val_acc: 0.8749356664951107 - bacc_train: 0.8475474488538328 - bacc_test: 0.8896809777088223 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 951 / 1000 
 - time: 0.02683544158935547 - sq_loss: 0.025370261792775444 - tot_loss: 0.4397781085202706 - acc: 0.8475474488538329 - val_acc: 0.8748909127525789 - bacc_train: 0.8475474488538328 - bacc_test: 0.8896579420726931 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 952 / 1000 
 - time: 0.026883602142333984 - sq_loss: 0.0252185060423737

Epoch 981 / 1000 
 - time: 0.026950836181640625 - sq_loss: 0.021596492765947252 - tot_loss: 0.42807928375877813 - acc: 0.8475474488538329 - val_acc: 0.8744433753272618 - bacc_train: 0.8475474488538328 - bacc_test: 0.8890478672227375 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 982 / 1000 
 - time: 0.026906967163085938 - sq_loss: 0.02149436092476948 - tot_loss: 0.42773457565749934 - acc: 0.8475474488538329 - val_acc: 0.8744209984559959 - bacc_train: 0.8475474488538328 - bacc_test: 0.8890363494046729 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 983 / 1000 
 - time: 0.02681708335876465 - sq_loss: 0.021393508001515456 - tot_loss: 0.42742591334912694 - acc: 0.8477939364062115 - val_acc: 0.87439862158473 - bacc_train: 0.8477939364062115 - bacc_test: 0.8890248315866085 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8979620853080569
Epoch 984 / 1000 
 - time: 0.0269467830657959 - sq_loss: 0.021293923660904

### Visualization of training results

In [ ]:
plt.figure()
plt.plot(np.arange(1,niter+1), loss2)
plt.yscale('log',base=2)
plt.title('training loss')

plt.figure()
plt.plot(np.arange(1,niter+1), accuracy_train)
plt.plot(np.arange(1,niter+1), accuracy_test)
plt.title('accuracy')